In [1]:
import numpy as np
import pandas as pd
from random import randrange
import io
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

from sklearn import metrics

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import SGDClassifier

from subprocess import check_output
print(check_output(["ls", "/Users/Hermione/MasterUCL/Web economics/Assignment/dataset"]).decode("utf8"))

test.csv
train.csv
validation.csv



In [2]:
train = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/train.csv")
test = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/test.csv")
validation = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/validation.csv")
train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'bidprice', 'payprice', 'keypage',
       'advertiser', 'usertag'],
      dtype='object')

In [3]:
columns_to_transform = ['weekday', "hour", "useragent",
        "region", "adexchange",   
        "slotwidth", "slotheight",
       "slotprice",
       "advertiser"]

df = pd.DataFrame(train[columns_to_transform])

In [4]:
df

,weekday,hour,useragent,region,adexchange,slotwidth,slotheight,slotprice,advertiser
0,1,14,windows_ie,80,2,468,60,5,1458
1,2,20,windows_chrome,94,2,250,250,5,3476
2,5,9,windows_ie,216,1,336,280,0,1458
3,2,20,windows_ie,216,2,728,90,162,3427
4,6,9,windows_chrome,40,1,950,90,0,3476
5,3,22,windows_ie,3,1,300,250,0,3358
6,5,16,windows_chrome,216,2,728,90,5,2259
7,0,21,windows_chrome,216,4,960,90,0,2821
8,4,17,windows_ie,201,2,336,280,5,3386
9,0,17,windows_ie,55,3,1000,90,31,1458


In [5]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
                
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
                
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [6]:
def split_useragent(data):
    data["OS"] = data['useragent'].apply(lambda x: x.split("_")[0])
    data["browser"]=data['useragent'].apply(lambda x: x.split("_")[1])
   


In [7]:
split_useragent(df)


In [8]:
def slot_price(column):
    slotprice = int(column)
    if slotprice >= 300:
        return '300+'
    elif slotprice >=200:
        return '200+'
    elif slotprice >=100:
        return '100+'
    elif slotprice >=50:
        return '50+'
    else:
        return "50-"

In [9]:
df["slotprice"] = df['slotprice'].apply(lambda x: slot_price(x))

In [10]:
df = MultiColumnLabelEncoder(columns = [ "OS","browser",
        "region", "adexchange",   
        "slotwidth", "slotheight",
       "slotprice",
       "advertiser"]).fit_transform(df)

In [11]:
df = df.drop('useragent',1)


In [26]:
def encode_onehot(df, cols):
    
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(orient='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df

In [20]:
df

,weekday,hour,region,adexchange,slotwidth,slotheight,slotprice,advertiser,OS,browser
0,1,14,10,1,11,1,4,0,5,2
1,2,20,11,1,6,10,4,8,5,0
2,5,9,19,0,9,11,4,0,5,2
3,2,20,19,1,14,3,0,7,5,2
4,6,9,6,0,17,3,4,8,5,0
5,3,22,3,0,7,10,4,5,5,2
6,5,16,19,1,14,3,4,1,5,0
7,0,21,19,3,18,3,4,3,5,0
8,4,17,18,1,9,11,4,6,5,2
9,0,17,7,2,20,3,4,0,5,2


In [27]:
df = encode_onehot(df, cols=['region', 'adexchange','slotwidth','slotheight','slotprice','advertiser','OS','browser'])

In [28]:
# Train the classifier
clf = SGDClassifier(loss="log")

#all_classes = np.array([0, 1])

y_train = train["click"]
    
Xcat = df

clf.fit(Xcat, y_train)
    
# Create a submission file
X_test = pd.read_csv("/Users/Hermione/MasterUCL/Web economics/Assignment/dataset/test.csv")
Trans = pd.DataFrame(test[columns_to_transform])
split_useragent(Trans)
Trans = Trans.drop('useragent',1)

X_enc_test = MultiColumnLabelEncoder(columns = [ "OS","browser",
        "region", "adexchange",   
        "slotwidth", "slotheight",
       "slotprice",
       "advertiser"]).fit_transform(Trans)

y_pred = clf.predict(X_enc_test)




In [30]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [31]:
sum(y_pred)

0